In [37]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [49]:
class Node:
    def __init__(self, feature=None, f_value=None, childs=None, label=None):
        self.feature = feature
        self.f_value = f_value
        self.childs = childs
        self.label = label
        
    def is_leaf_node(self):
        return self.label is not None

In [50]:
class DecisionTree:
    def __init__(self, min_samples_split=1):
        self.min_samples_split=min_samples_split
        self.root=None

    def fit(self, X, y):
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, f_value=None):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(f_value=f_value, label=leaf_value)

        # find the best split
        best_feature = self._best_split(X, y)

        # create child nodes
        childs = {}
        best_column = X[:, best_feature]
        for v in np.unique(best_column):
            index = np.argwhere(best_column == v).flatten()
            child_X = X[index]
            child_y = y[index]
            childs[v] = self._grow_tree(child_X, child_y, v)
        
        return Node(feature=best_feature, f_value=f_value, childs=childs)


    def _best_split(self, X, y):
        best_gain = -1
        split_idx = None

        for feat_idx in range(X.shape[1]):
            X_column = X[:, feat_idx]
            
            gain = self._information_gain(y, X_column)
            
            if gain > best_gain:
                best_gain = gain
                split_idx = feat_idx

        return split_idx


    def _information_gain(self, y, X_column):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        childs_idx = self._split(X_column)
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        information_gain = parent_entropy
#         print('an')
        for ch_id in childs_idx:
            n_ch = len(ch_id)
#             print(n_ch,'/', n)
            information_gain -= (n_ch / n) * self._entropy(y[ch_id])
            
        return information_gain

    def _split(self, X_column):
        splits = []
        for v in np.unique(X_column):
            splits.append(np.argwhere(X_column == v).flatten())
        return splits

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.label

        return self._traverse_tree(x, node.childs[x[node.feature]])
        

# Test Bed

In [4]:
import pandas as pd

data = {
    'alt': ['Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes'],
    'bar': ['No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes'],
    'fri': ['No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes'],
    'hun': ['Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes'],
    'pat': ['Some', 'Full', 'Some', 'Full', 'Full', 'Some', 'None', 'Some', 'Full', 'Full', 'None', 'Full'],
    'price': ['$$$', '$', '$', '$', '$$$', '$$', '$', '$$', '$', '$$$', '$', '$'],
    'rain': ['No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No'],
    'res': ['Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No'],
    'type': ['French', 'Thai', 'Burger', 'Thai', 'French', 'Italian', 'Burger', 'Thai', 'Burger', 'Italian', 'Thai', 'Burger'],
    'est': ['0-10', '30-60', '0-10', '10-30', '>60', '0-10', '0-10', '0-10', '>60', '10-30', '0-10', '30-60'],
    'will_wait': ['Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes']
}
df = pd.DataFrame(data)
df

,alt,bar,fri,hun,pat,price,rain,res,type,est,will_wait
0,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,Yes,No,Yes,Yes,Full,$,Yes,No,Thai,10-30,Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No
5,No,Yes,No,Yes,Some,$$,Yes,Yes,Italian,0-10,Yes
6,No,Yes,No,No,None,$,Yes,No,Burger,0-10,No
7,No,No,No,Yes,Some,$$,Yes,Yes,Thai,0-10,Yes
8,No,Yes,Yes,No,Full,$,Yes,No,Burger,>60,No
9,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,10-30,No


In [72]:
d1 = df[df['pat'] == 'Full']
d1

,alt,bar,fri,hun,pat,price,rain,res,type,est,will_wait
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
3,Yes,No,Yes,Yes,Full,$,Yes,No,Thai,10-30,Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No
8,No,Yes,Yes,No,Full,$,Yes,No,Burger,>60,No
9,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,10-30,No
11,Yes,Yes,Yes,Yes,Full,$,No,No,Burger,30-60,Yes


In [77]:
d2 = d1[ d1['type']=='Thai']
d2

,alt,bar,fri,hun,pat,price,rain,res,type,est,will_wait
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
3,Yes,No,Yes,Yes,Full,$,Yes,No,Thai,10-30,Yes


In [5]:
les = []
encoded_data = []
for column in list(df.columns):
    les.append(LabelEncoder().fit(df[column]))
    encoded_data.append(les[-1].transform(df[column]))
    
encoded_data = np.array(encoded_data).T
print(encoded_data)

[[1 0 0 1 2 2 0 1 1 0 1]
 [1 0 0 1 0 0 0 0 3 2 0]
 [0 1 0 0 2 0 0 0 0 0 1]
 [1 0 1 1 0 0 1 0 3 1 1]
 [1 0 1 0 0 2 0 1 1 3 0]
 [0 1 0 1 2 1 1 1 2 0 1]
 [0 1 0 0 1 0 1 0 0 0 0]
 [0 0 0 1 2 1 1 1 3 0 1]
 [0 1 1 0 0 0 1 0 0 3 0]
 [1 1 1 1 0 2 0 1 2 1 0]
 [0 0 0 0 1 0 0 0 3 0 0]
 [1 1 1 1 0 0 0 0 0 2 1]]


In [87]:
his = [[], [], [], [], []]
def traverse(root, depth=0):
    if root.feature is not None:
        his[depth].append((root.f_value, list(df.columns)[root.feature]))
    else:
        his[depth].append((root.f_value, 'label'+str(root.label)))
    if root.childs is not None:
        for child_key in root.childs:
            traverse(root.childs[child_key], depth + 1)

In [86]:
tree = DecisionTree()
tree.fit(encoded_data[:, :-1], encoded_data[:, -1])

In [79]:
y_pred = tree.predict(encoded_data[:, :-1])
print(accuracy_score(y_pred, encoded_data[:, -1]))

1.0


In [88]:
traverse(tree.root)
for i, li in enumerate(his):
    print('depth', i, li)

depth 0 [(None, 'pat')]
depth 1 [(0, 'type'), (1, 'label0'), (2, 'label1')]
depth 2 [(0, 'alt'), (1, 'label0'), (2, 'label0'), (3, 'fri')]
depth 3 [(0, 'label0'), (1, 'label1'), (0, 'label0'), (1, 'label1')]
depth 4 []
